# 07_Manual Energy Measurement Flashing

This script flashes the board with the selected settings and exports them as a json.
This json then can be used as a comment in the RocketLogger measurement to allow for an automatic analysis in the evaluation.

In [1]:
%%capture
# pick model name and then run the corresponding notebooks

#model_name = 'LeNet-MNIST'
model_name = '01d_ResNet20_CIFAR-10'

%run '03_TFLu-on-MCU.ipynb'

In [2]:
import json
from time import sleep
# Pyperclip is a cross-platform Python module for copy and paste clipboard functions.
import pyperclip

#### Use mbed to detect connected boards

In [3]:
!mbed detect

[mbed] Working path "/Users/nope/ownCloud/projects/masterthesis/jupyter-workspace/LeNet5-on-MNIST" (program)
[mbed] WARNING: The mbed-os tools were not found in "/Users/nope/ownCloud/projects/masterthesis/jupyter-workspace/LeNet5-on-MNIST". 
       Limited information will be shown about connected targets/boards
---
[mbed] Detected "DISCO_F469NI" connected to "/Volumes/DIS_F469NI" and using com port "/dev/tty.usbmodem145103"


### Define constants

Check the results from `mbed detect` for the serial port and device name.

In [6]:
target_device = "DISCO_F469NI"
serial_port = '/dev/tty.usbmodem145103'

arena_size_kb = 240

In [7]:
# all the available models
tfl_model_files = glob.glob(f'./TFLite-model/*{model_name}*.tflite')

baudrate = 1000000

mbed_none_cmsis_dir = './TFLu_benchmark-model_mbed'
mbed_cmsis_dir = './TFLu_benchmark-model_mbed_cmsis-nn'

np.random.seed(42) 
sample_images = np.random.randint(10000, size=200)
print(f"Using the following images indices from the testset:\n{sample_images}.")

mbed_dirs = {"none": mbed_none_cmsis_dir, 
             "cmsis-nn": mbed_cmsis_dir}

Using the following images indices from the testset:
[7270  860 5390 5191 5734 6265  466 4426 5578 8322 1685  769 6949 2433
 5311 5051 6420 1184 4555 3385 6396 8666 9274 2558 7849 2047 2747 9167
 9998  189 2734 3005 4658 1899 7734 1267 1528 3556 3890 8838 5393 8792
 8433 7513 2612 7041 9555 6235 5486 7099 9670  775 8226 3152 1585 3943
 7555 3073 1021 3843 7989 9692 6873 5675  161 4297  995 7629 9467 1016
 7869 6439 7892 6863 7916 8529  878 9268 4887 4859 6331 8571 8684 7208
 5276 2062   64 8006 2568 5463 2027 2695 9687 5258 5618 6736  391 5892
 3561 6184 3099 6278 8392 3104 7215 2454 8996 2731 8154 9762 5056 8110
 3840 1028 7385  502 6910 9062 6938 4488  206 5134 5977 7721 7035 1484
 7858  863 2790 7408 8755 5116 6019 1757 7574 6374 6892 1678 3242 4636
 1059 6668 9914 3157 5915 9789 2693 3627 9555 5450 1663 9721 5592 7392
 1306 6776 5864 9474 7526 8901 5575 5530 4413 3748  663 1998 7994 1495
 3304 3763 5232 1853 6585 1291 3581 7554 7280 1636 3696  698 4737  854
 8164 5855 7392 6528 524

In [8]:
# select mcu
mcu_select = widgets.Dropdown(
    options=["NUCLEO_L496ZG", "NUCLEO_F767ZI", "DISCO_F469NI"],
    description='mbed:',
    layout=Layout(width='100%')
)
display(mcu_select)

Dropdown(description='mbed:', layout=Layout(width='100%'), options=('NUCLEO_L496ZG', 'NUCLEO_F767ZI', 'DISCO_F…

In [9]:
target_device = mcu_select.value
target_device

'DISCO_F469NI'

In [10]:
file_select = widgets.Dropdown(
    options=tfl_model_files,
    description='Models: ',
    layout=Layout(width='100%')
)
display(file_select)

Dropdown(description='Models: ', layout=Layout(width='100%'), options=('./TFLite-model/01d_ResNet20_CIFAR-10_n…

In [11]:
tfl_model_file = file_select.value
tfl_model_file

'./TFLite-model/01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none.tflite'

In [12]:
mbed_select = widgets.Dropdown(
    options=["none", "cmsis-nn"],
    description='mbed:',
    layout=Layout(width='100%')
)
display(mbed_select)

Dropdown(description='mbed:', layout=Layout(width='100%'), options=('none', 'cmsis-nn'), value='none')

In [17]:
mbed_dir = mbed_select.value
mbed_dir

'cmsis-nn'

### Run the layer benchmark for all the selected models

The json including all the model information will be printed and automatically stored in your clipboard.

In [18]:
fpu_status = 1
compiler_flags = ['-Ofast', '-Os']
compiler_flag = '-Ofast'


model_information = {}
model_information['time'] = str(datetime.now())
model_information['MCU'] = target_device

tfl_model_string = model_string(tfl_model_file)
model_information['model'] = tfl_model_string
model_information['model_name'] = model_name

print("\tUsing the following mbed:", mbed_dirs[mbed_dir])
model_information['mbed-dir'] = mbed_dirs[mbed_dir]
print("\tUsing the following software library:", mbed_dir)
model_information['cmsis-nn'] = mbed_dir

patch_arena_size(mbed_dirs[mbed_dir], arena_size_kb)

read_model_information(tfl_model_string, model_information)

model_information['model_type'] = 'W-' + model_information['weights'] + '_A-' +model_information['activations']


tfl_model_to_file(tfl_model_file, mbed_dirs[mbed_dir])

write_constants(tfl_model_string, 1, image_no, mbed_dirs[mbed_dir])

set_compiler_flag(mbed_dirs[mbed_dir], compiler_flag)
model_information['compiler_optimization'] = compiler_flag

toggle_fpu(mbed_dirs[mbed_dir], fpu_status)
model_information['FPU_status'] = fpu_status

print("\tBuilding & flashing ...")
!cd {mbed_dirs[mbed_dir]} && mbed compile -t GCC_ARM --profile release -D BENCHMARK_LAYERS -D INPUT_LENGTH={INPUT_LENGTH} -D ENERGY_MEASUREMENT -D NO_REPORTING --flash > build_log.txt

if _exit_code != 0:
    print("\tThere was an error during compilation. Skipping ...")
    model_information['error'] = 'error during compilation'

print("\tFinished building & flashing.")

# print json
print()
print("Use the following json as a comment for the RocketLogger:")
print("(It's  already in your clipboard)\n")

json_dump = json.dumps(model_information, indent=2)
print(json_dump)
pyperclip.copy(json_dump)

	Using the following mbed: ./TFLu_benchmark-model_mbed_cmsis-nn
	Using the following software library: cmsis-nn
	Building & flashing ...
	Finished building & flashing.

Use the following json as a comment for the RocketLogger:
(It's  already in your clipboard)

{
  "time": "2020-08-07 14:52:35.576954",
  "MCU": "DISCO_F469NI",
  "model": "01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none",
  "model_name": "01d_ResNet20_CIFAR-10",
  "mbed-dir": "./TFLu_benchmark-model_mbed_cmsis-nn",
  "cmsis-nn": "cmsis-nn",
  "pruned": 0,
  "weights": "float32",
  "activations": "float32",
  "model_type": "W-float32_A-float32",
  "compiler_optimization": "-Ofast",
  "FPU_status": 1
}


## Send sample images and run the measurement with the RocketLogger in the meantime

In [19]:
serial_port = '/dev/tty.usbmodem145103'

# the sleep time between sending new images for inference must be manually adjusted
# as the MCU doesn't report the inference results back - we do not know when the inference is finished

# L4: 0.5is fine for LeNet
# ResNet20
# L4: 4s for int8 cmsis-nn
# L4: 15s for int8 none

sleep_time = 8

In [20]:
port = set_port(serial_port, baudrate)


print(f"Benchmarking layers (sample of {len(sample_images)} images) ...")
print("Doing a single inference ...")
send_image(port, x_test_normalized[image_no])
sleep(sleep_time)
print("Now start the measurement! HURRY!")
sleep(1)
print("\t5")
sleep(1)
print("\t4")
sleep(1)
print("\t3")
sleep(1)
print("\t2")
sleep(1)
print("\t1")
sleep(1)
print("Here we go ...")


for i, image_no in enumerate(sample_images):
    if(i % 10 == 0):
        print(f"\t{int(100 * i/sample_images.shape[0])}%")
    send_image(port, x_test_normalized[image_no])
    # can't read results they're not reported
    # _, benchmark_result = read_results(port)
    sleep(sleep_time)

print("Fin.")
print("Now manually stop the measurement and save the file.")
port.close()

Benchmarking layers (sample of 200 images) ...
Doing a single inference ...
Now start the measurement! HURRY!
	5
	4
	3
	2
	1
Here we go ...
	0%
	5%
	10%
	15%
	20%
	25%
	30%
	35%
	40%
	45%
	50%
	55%
	60%
	65%
	70%
	75%
	80%
	85%
	90%
	95%
Fin.
Now manually stop the measurement and save the file.


No manually download the file an run the analysis with `ML-on-MCU_data-analysis/03a_`